**Biomedical Software Engineering**

**Prof. Arthur Goldberg**

**Dept. Genetics and Genomic Sciences**

**Spring 1, 2020**

# Python unit testing

In [0]:
import sys
print(sys.version)

3.6.9 (default, Nov  7 2019, 10:44:02) 
[GCC 8.3.0]


## Basics of unittest

### Exceptions

In [0]:
class Error(Exception):
    """ Base class for exceptions in this module
    """
    pass

class PersonError(Error):
    """ Exception raised for errors in this module

    Attributes:
        message -- explanation of the error
    """
    def __init__(self, message):
        self.message = message

### Code to test
Let's test a class that represents male or female gender. (Lines have been compressed to fit more on a screen.)

In [0]:
class Gender(object):
    """ Gender for a person """
    # TODO: incorporate other genders
    MALE = 'M'
    FEMALE = 'F'
    UNKNOWN = 'unknown'
    def __init__(self):
        # '1' and '2' map to male and female in PED files
        self.gender_map = {
            self.MALE: set(['male', 'm', '1']),
            self.FEMALE: set(['female', 'f', '2']),
            self.UNKNOWN: set(['unknown', 'na', 'not specified', '-9', '0'])}

    def get_gender(self, gender):
        """ Convert a string into a gender constant
        Args:
             gender (:obj:`str`): a gender value
        Returns:
            :obj:`str`: a reference gender value, stored in a constant value in this class
        Raises:
            :obj:`PersonError`: if `gender` does not map to a reference gender value
        """
        for gender_constant, synonyms in self.gender_map.items():
            if gender.lower() in synonyms:
                return gender_constant
        raise PersonError(f"Illegal gender '{gender}'")

    def genders_string_mappings(self):
        """ Report the mappings from strings to gender constants
        Returns:
            :obj:`str`: a description of the mappings from strings to gender constants
        """
        rv = "Legal genders, which are case insensitive, map to gender constants:\n"
        for gender_constant,synonyms in self.gender_map.items():
            rv += "{} -> '{}'\n".format(synonyms, gender_constant)
        return rv


### Test `Genders`

In [0]:
import unittest
# outside Jupyter will need to import the code being tested
# from answer_person import Person, Gender, PersonError

# Tests are written in subclasses of TestCase
class TestGender(unittest.TestCase):

    # each individual test is a function that starts with 'test'
    def test_gender(self):
        # self.assertEqual(actual, expected) checks that actual == expected
        self.assertEqual(Gender().get_gender('Male'), Gender.MALE)
        self.assertEqual(Gender().get_gender('female'), Gender.FEMALE)
        self.assertEqual(Gender().get_gender('FEMALE'), Gender.FEMALE)
        self.assertEqual(Gender().get_gender('NA'), Gender.UNKNOWN)
        # test numeric representations
        self.assertEqual(Gender().get_gender('1'), Gender.MALE)
        self.assertEqual(Gender().get_gender('2'), Gender.FEMALE)
        self.assertEqual(Gender().get_gender('0'), Gender.UNKNOWN)
        self.assertEqual(Gender().get_gender('-9'), Gender.UNKNOWN)

        # self.assertRaises checks that an exception is raised
        with self.assertRaises(PersonError):
          Gender().get_gender('not a gender')

        # assertRaisesRegex also checks the content of the exception
        with self.assertRaisesRegex(PersonError, "Illegal gender"):
          Gender().get_gender('not a gender')

        # assertRegex(actual, pattern) checks that pattern.search(actual) holds
        self.assertRegex(Gender().genders_string_mappings(), "'f'.* -> 'F'")

# code to run a unittest in Jupyter
def run_test_class(cls):
  suite = unittest.TestLoader().loadTestsFromModule(cls)
  unittest.TextTestRunner().run(suite)

run_test_class(TestGender())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### Many other assert functions are available:

<img src="https://drive.google.com/uc?id=1j6i6o5WKZp-MSiKzSyhNTOkLtRCa2vJi" width="500px"/>

Let's try a few.




In [0]:
# Here's a Person class
class Person(object):
    """ Person

    Attributes:
        name (:obj:`str`): a person's name
        gender (:obj:`str`): a person's gender, which must be an attribute of `Gender`
        mother (:obj:`Person`): a person's mother
        father (:obj:`Person`): a person's father
        children (:obj:`set` of `Person`): a person's children
    """

    def __init__(self, name, gender, mother=None, father=None):
        """ Create a Person instance

        Create a Person instance. This is used by the expression Person().
        The parameters name and gender are required, while other parameters are optional.

        Args:
             name (:obj:`str`): the person's name
             gender (:obj:`str`): the person's gender
             mother (:obj:`Person`, optional): the person's mother
             father (:obj:`Person`, optional): the person's father

        Raises:
            :obj:`PersonError`: if `gender` does not map to a reference gender value
        """
        self.name = name
        self.gender = Gender().get_gender(gender)
        self.mother = mother
        self.father = father
        self.children = set()

    def __repr__(self):
        """ Provide a string representation of this person"""
        return "<Person at {}: name: {}; gender: {}>".format(
            str(id(self)),
            self.name,
            self.gender
            )

    def __str__(self):
        '''A representation of a Person object'''
        return self.__repr__()

    @staticmethod
    def get_persons_name(person):
        """ Get a person's name; if the person is not known, return 'NA'

        Returns:
            :obj:`str`: the person's name, or 'NA' if they're not known
        """
        if person is None:
            return 'NA'
        return person.name

    def set_mother(self, mother):
        """ Set the mother of this person

        Args:
             mother (:obj:`Person`): this person's mother

        Raises:
            :obj:`PersonError`: if `mother` is not female, or if a cycle in the ancestors
            graph would be created
        """
        if mother.gender != Gender.FEMALE:
            raise PersonError("mother named '{}' is not female".format(mother.name))
        mother.children.add(self)
        self.mother = mother

    def set_father(self, father):
        """ Set the father of this person

        Args:
             father (:obj:`Person`): this person's father

        Raises:
            :obj:`PersonError`: if `father` is not male, or if a cycle in the ancestors
            graph would be created
        """
        if father.gender != Gender.MALE:
            raise PersonError("father named '{}' is not male".format(father.name))
        father.children.add(self)
        self.father = father

    def add_child(self, child):
        """ Add a child to this person's children, and set this person as the child's father or mother

        Args:
             child (:obj:`Person`): a child of `self`

        Raises:
            :obj:`PersonError`: if this person does not have a known gender, or if a cycle in the
            ancestors graph would be created
        """
        if self.gender not in [Gender.FEMALE, Gender.MALE]:
            raise PersonError("cannot add child to person named '{}' with unknown gender".format(
                self.name))
        if child in self.all_ancestors():
            raise PersonError("making '{}' a child of '{}', would create ancestor cycle".format(
                child.name, self.name))
        if self.gender == Gender.FEMALE:
            child.set_mother(self)
        if self.gender == Gender.MALE:
            child.set_father(self)

    def remove_father(self):
        """ Remove this person's father

        Raises:
            :obj:`PersonError`: if this person does not have a father or this person is not one
            of their father's children
        """
        if not isinstance(self.father, Person):
            raise PersonError("cannot remove father of '{}', as it is not set".format(self.name))
        if not self in self.father.children:
            raise PersonError("cannot remove father of '{}', not one of his children".format(self.name))
        self.father.children.remove(self)
        self.father = None

    def remove_mother(self):
        """ Remove this person's mother

        Raises:
            :obj:`PersonError`: if this person does not have a mother or this person is not one
            of their mother's children
        """
        if not isinstance(self.mother, Person):
            raise PersonError("mother of '{}' is not set and cannot be removed".format(self.name))
        if not self in self.mother.children:
            raise PersonError("cannot remove mother of '{}', not one of her children".format(self.name))
        self.mother.children.remove(self)
        self.mother = None

    def ancestors(self, min_depth, max_depth=None):
        """ Return this person's ancestors within a generational depth range

        Obtain ancestors whose generational depth satisfies `min_depth` <= depth <= `max_depth`. E.g.,
        a person's parents would be obtained with `min_depth` = 1, and this person's parents and
        grandparents would be obtained with `min_depth` = 1 and `max_depth` = 2.

        Args:
            min_depth (:obj:`int`): the minimum depth of ancestors which should be provided;
                this person's depth is 0, their parents' depth is 1, etc.
            max_depth (:obj:`int`, optional): the minimum depth of ancestors which should be
                provided; if `max_depth` is not provided, then `max_depth` == `min_depth` so that only
                ancestors at depth == `min_depth` will be provided; a `max_depth` of infinity will obtain
                all ancestors at depth >= `min_depth`.

        Returns:
            :obj:`set` of `Person`: this person's ancestors

        Raises:
            :obj:`PersonError`: if `max_depth` < `min_depth`
        """
        if max_depth is not None:
            if max_depth < min_depth:
                    raise PersonError("max_depth ({}) cannot be less than min_depth ({})".format(
                        max_depth, min_depth))
        else:
            # collect just one depth
            max_depth = min_depth
        collected_ancestors = set()
        return self._ancestors(collected_ancestors, min_depth, max_depth)

    def _ancestors(self, collected_ancestors, min_depth, max_depth):
        """ Obtain this person's ancestors who lie within the generational depth [min_depth, max_depth]

        This is a private, recursive method that recurses through the ancestry via parent references.

        Args:
            collected_ancestors (:obj:`set`): ancestors collected thus far by this method
            min_depth (:obj:`int`): see `ancestors()`
            max_depth (:obj:`int`): see `ancestors()`

        Returns:
            :obj:`set` of `Person`: this person's ancestors

        Raises:
            :obj:`PersonError`: if `max_depth` < `min_depth`
        """
        if min_depth <= 0:
            collected_ancestors.add(self)
        if 0 < max_depth:
            for parent in [self.mother, self.father]:
                if parent is not None:
                    parent._ancestors(collected_ancestors, min_depth-1, max_depth-1)
        return collected_ancestors

    def parents(self):
        ''' Provide this person's parents

        Returns:
            :obj:`set`: this person's known parents
        '''
        return self.ancestors(1)

    def grandparents(self):
        ''' Provide this person's known grandparents, by using ancestors()

        Returns:
            :obj:`set`: this person's known grandparents
        '''
        return self.ancestors(2)

    def all_grandparents(self):
        ''' Provide all of this person's known grandparents, from their parents' parents on back

        Returns:
            :obj:`set`: all of this person's known grandparents
        '''
        return self.ancestors(2, max_depth=float('inf'))

    def all_ancestors(self):
        ''' Provide all of this person's known ancestors

        Returns:
            :obj:`set`: all of this person's known ancestors
        '''
        return self.ancestors(1, max_depth=float('inf'))

### Test parts of it

In [0]:
class TestPerson(unittest.TestCase):

    # setUp runs before EACH individual test
    def setUp(self):
        # instance attributes set by setUp() are available to all individual tests
        # create a few Persons
        self.child = Person('kid', 'NA')
        self.mom = Person('mom', 'f')
        self.dad = Person('dad', 'm')

    def test_set_mother(self):
        # assertTrue(x) checks that bool(x) is True
        self.assertTrue(self.mom.gender is Gender.FEMALE)

        self.child.set_mother(self.mom)
        self.assertEqual(self.child.mother, self.mom)
        self.assertIn(self.child, self.mom.children)

        # assertIsInstance(a, b) checks that isinstance(a, b) is True
        self.assertIsInstance(self.child.mother, Person)

        self.mom.gender = Gender.MALE
        with self.assertRaises(PersonError):
            self.child.set_mother(self.mom)

    def test_add_child(self):
        self.assertNotIn(self.child, self.mom.children)
        self.mom.add_child(self.child)
        self.assertEqual(self.child.mother, self.mom)
        self.assertIn(self.child, self.mom.children)

        self.assertNotIn(self.child, self.dad.children)
        self.dad.add_child(self.child)
        self.assertEqual(self.child.father, self.dad)
        self.assertIn(self.child, self.dad.children)

        # assertEqual can compare containers, like sets, lists, dictionaries, ...
        self.assertEqual(self.mom.children, self.dad.children)
        
        list1 = 'we love Monty Python'.split()
        self.assertEqual(list1, list1)
        self.assertNotEqual(list1, sorted(list1))
        self.assertEqual(list1, list1[:-1])

run_test_class(TestPerson())

F.
FAIL: test_add_child (__main__.TestPerson)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-31-dc2e52138b2b>", line 43, in test_add_child
    self.assertEqual(list1, list1[:-1])
AssertionError: Lists differ: ['we', 'love', 'Monty', 'Python'] != ['we', 'love', 'Monty']

First list contains 1 additional elements.
First extra element 3:
'Python'

- ['we', 'love', 'Monty', 'Python']
?                       ----------

+ ['we', 'love', 'Monty']

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (failures=1)


In [0]:
class TestPerson2(unittest.TestCase):

    # setUp can do anything
    def setUp(self):

        # make a deep family history
        self.generations = 4
        self.people = people = []
        self.root_child = Person('root_child', Gender.UNKNOWN)
        people.append(self.root_child)

        # add_parents() recursively creates ancestors back max_depth generations
        def add_parents(child, depth, max_depth):
            if depth+1 < max_depth:
                dad = Person(child.name + '_dad', Gender.MALE)
                mom = Person(child.name + '_mom', Gender.FEMALE)
                people.append(dad)
                people.append(mom)
                child.set_father(dad)
                child.set_mother(mom)
                add_parents(dad, depth+1, max_depth)
                add_parents(mom, depth+1, max_depth)
        add_parents(self.root_child, 0, self.generations)

    def test_all_ancestors(self):
      # all_ancestors() returns all of a person's ancestors
      self.assertEqual(self.root_child.all_ancestors(), set(self.people[1:]))

run_test_class(TestPerson2())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### test *fixture*
A test *fixture* is the term used to describe the preparation for a test and any cleanup needed after a test. 
`setUp` can create fixtures that get initialized before each test method. `tearDown` cleans up the fixture after the test runs.

In [0]:
import tempfile
import shutil

class TestExample(unittest.TestCase):

    def setUp(self):
        self.tempdir = tempfile.mkdtemp()

    # tearDown is called immediately after the test method; called even if the test
    # method raised an exception
    # tearDown is typically used to deallocate resources, as in this commoon pattern
    def tearDown(self):
        shutil.rmtree(self.tempdir)
        pass

    def test1(self):
      print(self.tempdir)

    def test2(self):
      print(self.tempdir)

run_test_class(TestExample())

E.

/tmp/tmpnbnmrcy4
/tmp/tmp6f387hj3



ERROR: test1 (__main__.TestExample)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-33-5491bac0488e>", line 18, in test1
    1 / 0
ZeroDivisionError: division by zero

----------------------------------------------------------------------
Ran 2 tests in 0.003s

FAILED (errors=1)


In [0]:
import os
print(os.path.isdir('/tmp'))
os.listdir('/tmp')

True


['tmpiljwl5s2', 'tmpl1pke9x2']

### decorators of unittest methods and classes
`@unittest.skip(message)` will skip a test method or `TestClass` and print the message.
`@unittest.expectedFailure` indicates that a test method should fail.

In [43]:
class TestExample2(unittest.TestCase):

  @unittest.expectedFailure
  def test_1(self):
      self.assertTrue(False)

  # careful: if multiple tests have the same name, the last runs & the others are silently ignored
  def test_2(self):
      print('\nhi mom')

  @unittest.skip('bad test')
  def test_bad(self):
      self.assertEqual(False, True)

run_test_class(TestExample2())

x.s


hi mom



----------------------------------------------------------------------
Ran 3 tests in 0.010s

OK (skipped=1, expected failures=1)
